In [3]:
import os
import numpy as np
import scipy.io.wavfile as wav
from scipy.fft import fft
from scipy.signal import find_peaks

# Função para extrair informações dos arquivos .txt (início, fim e classificação)
def extrair_informacoes_do_arquivo(caminho_arquivo):
    informacoes = []
    with open(caminho_arquivo, 'r') as arquivo:
        for linha in arquivo:
            partes = linha.strip().split('\t')
            if len(partes) >= 3:
                try:
                    inicio = float(partes[0])
                    fim = float(partes[1])
                    classificacao = partes[2]
                    informacoes.append((inicio, fim, classificacao))
                except ValueError:
                    continue
    return informacoes

# Função para calcular frequências em um intervalo de áudio com filtro de frequência
def calcular_frequencias_em_intervalo(caminho_arquivo_wav, inicio, fim, taxa_amostragem, faixa_frequencia=(5500, 10000)):
    # Carregar o arquivo .wav
    taxa, dados = wav.read(caminho_arquivo_wav)

    # Calcular o índice de amostra para os tempos de início e fim
    inicio_amostra = int(inicio * taxa_amostragem)
    fim_amostra = int(fim * taxa_amostragem)

    # Extrair o intervalo de áudio
    intervalo_audio = dados[inicio_amostra:fim_amostra]

    # Calcular a FFT do intervalo
    n = len(intervalo_audio)
    yf = fft(intervalo_audio)
    xf = np.fft.fftfreq(n, 1/taxa_amostragem)

    # Considerar apenas a parte positiva da frequência
    xf = xf[:n//2]
    yf = np.abs(yf[:n//2])

    # Aplicar o filtro de faixa de frequência
    mask = (xf >= faixa_frequencia[0]) & (xf <= faixa_frequencia[1])
    xf_filtrado = xf[mask]
    yf_filtrado = yf[mask]

    # Encontrar picos na FFT filtrada para calcular frequências
    picos, _ = find_peaks(yf_filtrado, height=0.1 * np.max(yf_filtrado))  # Ajuste o valor de altura conforme necessário
    frequencias_picos = xf_filtrado[picos]

    frequencia_media = np.mean(frequencias_picos) if len(frequencias_picos) > 0 else 0
    frequencia_maxima = np.max(frequencias_picos) if len(frequencias_picos) > 0 else 0
    frequencia_minima = np.min(frequencias_picos) if len(frequencias_picos) > 0 else 0

    return frequencia_media, frequencia_maxima, frequencia_minima

# Caminho para a pasta onde estão os arquivos .txt e .wav
pasta_txt = 'C:/Users/pedro/OneDrive/Documentos/teste/rotulos/'
pasta_wav = 'C:/Users/pedro/OneDrive/Documentos/teste/audios/'

# Taxa de amostragem padrão para áudio (ajuste conforme necessário)
taxa_amostragem = 96000  # 96 kHz, comum para arquivos WAV

# Processar todos os arquivos .txt e correspondentes .wav
for nome_arquivo_txt in os.listdir(pasta_txt):
    if nome_arquivo_txt.endswith('.txt'):
        nome_base = os.path.splitext(nome_arquivo_txt)[0]
        
        # Buscar tanto arquivos .wav quanto .WAV
        caminho_arquivo_wav = os.path.join(pasta_wav, nome_base + '.wav')
        if not os.path.exists(caminho_arquivo_wav):
            caminho_arquivo_wav = os.path.join(pasta_wav, nome_base + '.WAV')
        
        # Extrair todas as informações do arquivo .txt (início, fim, classificação)
        informacoes = extrair_informacoes_do_arquivo(os.path.join(pasta_txt, nome_arquivo_txt))

        if os.path.exists(caminho_arquivo_wav):
            # Criar um arquivo .txt para salvar as informações
            caminho_saida = os.path.join(pasta_txt, nome_base + '_frequencias.txt')
            with open(caminho_saida, 'w') as arquivo_saida:
                arquivo_saida.write('Nome do Áudio: {}\n\n'.format(nome_base + '.wav'))
                arquivo_saida.write('Início (s)\tFim (s)\t\tFrequência Máxima (Hz)\t\tFrequência Média (Hz)\t\tFrequência Mínima (Hz)\t\tClassificação\n')
                arquivo_saida.write('-------------------------------------------------------------------------------------------------------------\n')

                for inicio, fim, classificacao in informacoes:
                    frequencia_media, frequencia_maxima, frequencia_minima = calcular_frequencias_em_intervalo(
                        caminho_arquivo_wav, inicio, fim, taxa_amostragem)
                    arquivo_saida.write(f'{inicio:.4f}\t\t{fim:.4f}\t\t\t{frequencia_maxima:.2f}\t\t\t\t{frequencia_media:.2f}\t\t\t{frequencia_minima:.2f}\t\t\t\t{classificacao}\n')

                arquivo_saida.write('\n')  # Adicionar uma linha em branco entre os conjuntos de dados

            print(f'Informações salvas em: {caminho_saida}')


C:\Users\pedro\AppData\Local\Temp\ipykernel_7404\4235347800.py:26: WavFileWarning: Chunk (non-data) not understood, skipping it.
  taxa, dados = wav.read(caminho_arquivo_wav)


Informações salvas em: C:/Users/pedro/OneDrive/Documentos/teste/rotulos/20221028_052600_frequencias.txt
Informações salvas em: C:/Users/pedro/OneDrive/Documentos/teste/rotulos/20230213_160800_frequencias.txt
Informações salvas em: C:/Users/pedro/OneDrive/Documentos/teste/rotulos/20230213_161000_frequencias.txt
Informações salvas em: C:/Users/pedro/OneDrive/Documentos/teste/rotulos/20230402_054200_frequencias.txt
Informações salvas em: C:/Users/pedro/OneDrive/Documentos/teste/rotulos/20230413_140000_frequencias.txt
